In [2]:
import numpy as np

In [3]:
import os

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential

In [4]:
import os
import shutil

# define paths to the original client and imposter folders
path = r"C:\Users\PRANNOY\Downloads\raw\raw"
client_path = os.path.join(path, "ClientRaw")
imposter_path = os.path.join(path, "ImposterRaw")

# create new folders for train and test sets
train_path = os.path.join(path, "train")
test_path = os.path.join(path, "test")
os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)

# copy a portion of the images from the original folders to the train and test sets
for folder_name in os.listdir(client_path):
    folder_path = os.path.join(client_path, folder_name)
    if os.path.isdir(folder_path):
        images = os.listdir(folder_path)
        num_train_images = int(len(images) * 0.8)
        for i, image_name in enumerate(images):
            image_path = os.path.join(folder_path, image_name)
            if i < num_train_images:
                target_path = os.path.join(train_path, "client", folder_name)
            else:
                target_path = os.path.join(test_path, "client", folder_name)
            os.makedirs(target_path, exist_ok=True)
            shutil.copy2(image_path, target_path)

for folder_name in os.listdir(imposter_path):
    folder_path = os.path.join(imposter_path, folder_name)
    if os.path.isdir(folder_path):
        images = os.listdir(folder_path)
        num_train_images = int(len(images) * 0.8)
        for i, image_name in enumerate(images):
            image_path = os.path.join(folder_path, image_name)
            if i < num_train_images:
                target_path = os.path.join(train_path, "imposter", folder_name)
            else:
                target_path = os.path.join(test_path, "imposter", folder_name)
            os.makedirs(target_path, exist_ok=True)
            shutil.copy2(image_path, target_path)


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# create an ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# set the batch size
batch_size = 32

# set the paths to the train and test directories
train_path = r'C:\Users\PRANNOY\Downloads\raw\raw\train'
test_path = r'C:\Users\PRANNOY\Downloads\raw\raw\test'

# create the train and test generators
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary'
)
from tensorflow.keras.applications.vgg16 import VGG16

Found 10105 images belonging to 2 classes.
Found 2509 images belonging to 2 classes.


In [6]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# freeze the weights of the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# add a new classifier on top of the pre-trained layers
x = base_model.output
x = tf.keras.layers.Flatten()(x)

x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.models.Model(inputs=base_model.input, outputs=x)

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples/train_generator.batch_size,
    epochs=3,
    validation_data=test_generator,
    validation_steps=test_generator.samples/test_generator.batch_size)

Epoch 1/3
315/315 [==============================] - 3300s 10s/step - loss: 0.1101 - accuracy: 0.9685 - val_loss: 0.0863 - val_accuracy: 0.9486
Epoch 2/3
315/315 [==============================] - 23728s 75s/step - loss: 0.0277 - accuracy: 0.9913 - val_loss: 0.2835 - val_accuracy: 0.9330
Epoch 3/3
315/315 [==============================] - 2337s 7s/step - loss: 0.0266 - accuracy: 0.9905 - val_loss: 0.3048 - val_accuracy: 0.9334


In [7]:
test_loss, test_acc = model.evaluate(test_generator)

# print the test accuracy
print('Test accuracy:', test_acc)

79/79 [==============================] - 437s 6s/step - loss: 0.3048 - accuracy: 0.9334
Test accuracy: 0.9334396123886108


In [8]:
model.save('liveness_detection_model.h5')


In [9]:
print(os.path.exists('liveness_detection_model.h5'))

True


In [5]:
print(os.getcwd())

C:\Users\PRANNOY\Desktop\Jupyter notebooks


In [6]:
print(os.listdir())

['.ipynb_checkpoints', 'captured_images', 'captured_images_faak', 'captured_images_fake', 'captured_photos', 'liveness_detection.ipynb', 'liveness_detection_model.h5', 'loaded_model.ipynb', 'Making_work.ipynb', 'Untitled.ipynb', 'WEBCAM CLICKS.ipynb']
